In [1]:
import tushare as ts
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['KaiTi']
plt.rcParams['axes.unicode_minus'] = False
ts.set_token('5f74ec7e9d0e71e8b8174fc48550635013e084a53d63af4cac832ab3')
pro = ts.pro_api()

In [2]:
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

In [3]:
stock_list = data[['symbol']].values.T[0].tolist()

In [59]:
import pandas as pd

In [160]:
def get_end_volume(stock):
    df = ts.get_tick_data(stock, date='2019-11-26',src='tt')
    df_deal = df[df['time'] > '14:54:00']
    df1 = df_deal[-1:]
    df2 = df_deal[:-1]
    try:
        
        price1, v1, a1, bs_type = tuple(df1[['price', 'volume', 'amount', 'type']].values[0])
        v2, a2, price2 = df2.agg({'volume': 'sum', 'amount': 'sum', 'price':'max'})
        return pd.Series([price2, v2, a2, price1, v1, a1, bs_type])
    except Exception as e:
        return pd.Series([0, 0, 0, 0, 0, 0, '中性盘'])

In [161]:
stock_end = data['symbol'].apply(get_end_volume).rename(columns={0:'last_3min_max_price', 1:'last_3min_vol', 2:'last_3min_amount', 3:'bidding_price', 4:'bidding_vol', 5:'bidding_amount', 6:'买卖类型'})

In [165]:
df_res = pd.concat([data, stock_end] ,axis=1)
def get_change_rate(row):
    if row['last_3min_vol'] == 0:
        return  0
    return pd.Series([row['bidding_vol'] / row['last_3min_vol'], row['bidding_amount'] - row['last_3min_amount'], (1 - row['bidding_price'] / row['last_3min_max_price']) * -100 if row['last_3min_max_price'] != 0 else 0])
df_res = pd.concat([df_res, df_res.apply(get_change_rate, axis=1)], axis=1).rename(columns={0: 'change', 1:'rest_amount', 2:'pp'})

In [183]:
df1 = df_res[((abs(df_res['pp']) > 0.5) & (df_res['bidding_price'] > 2)) | (df_res['bidding_vol'] > 20000 )]

In [194]:
df_res.append(df1).drop_duplicates(keep=False)

,ts_code,symbol,name,area,industry,list_date,last_3min_max_price,last_3min_vol,last_3min_amount,bidding_price,bidding_vol,bidding_amount,买卖类型,change,rest_amount,pp
2,000004.SZ,000004,国农科技,深圳,生物制药,19910114,18.59,5.0,9295.0,18.66,22,41052,卖盘,4.400000,31757.0,0.376547
3,000005.SZ,000005,世纪星源,深圳,环境保护,19901210,2.97,1904.0,564960.0,2.97,3654,1085238,卖盘,1.919118,520278.0,-0.000000
4,000006.SZ,000006,深振业A,深圳,区域地产,19920427,5.15,711.0,365982.0,5.14,968,497069,卖盘,1.361463,131087.0,-0.194175
6,000008.SZ,000008,神州高铁,北京,运输设备,19920507,3.52,9392.0,3298836.0,3.51,1958,687258,卖盘,0.208475,-2611578.0,-0.284091
7,000009.SZ,000009,中国宝安,深圳,综合类,19910625,4.49,3671.0,1644517.0,4.47,567,253449,卖盘,0.154454,-1391068.0,-0.445434
8,000010.SZ,000010,*ST美丽,深圳,建筑工程,19951027,3.42,691.0,236322.0,3.42,51,17442,卖盘,0.073806,-218880.0,-0.000000
9,000011.SZ,000011,深物业A,深圳,区域地产,19920330,9.01,253.0,227758.0,8.97,685,614445,卖盘,2.707510,386687.0,-0.443951
11,000014.SZ,000014,沙河股份,深圳,全国地产,19920602,8.88,384.0,340569.0,8.84,477,421668,卖盘,1.242188,81099.0,-0.450450
12,000016.SZ,000016,深康佳A,深圳,家用电器,19920327,4.08,1254.0,510660.0,4.07,771,313797,卖盘,0.614833,-196863.0,-0.245098
13,000017.SZ,000017,深中华A,深圳,文教休闲,19920331,4.33,1752.0,757302.0,4.33,874,378442,卖盘,0.498858,-378860.0,-0.000000


In [196]:
df1.shape

(670, 16)

In [202]:
df1[df1['pp'] > 0].shape[0], df1[df1['pp'] < 0].shape[0]

(205, 449)

In [201]:
df1[df1['pp'] > 0].rest_amount.sum(), df1[df1['pp'] < 0].rest_amount.sum()

(16030850384.0, 11847747340.0)

In [211]:
df1[df1['pp'] < 0].groupby('industry').size().sort_values(ascending=False)

industry
软件服务    21
元器件     18
汽车配件    16
化学制药    14
区域地产    12
银行      12
医疗保健    12
电气设备    12
专用机械    11
通信设备    11
普钢      10
互联网     10
机械基件    10
化工原料    10
火力发电     8
煤炭开采     8
半导体      8
建筑工程     8
证券       7
生物制药     6
综合类      6
农药化肥     6
影视音像     6
环境保护     6
中成药      6
造纸       6
工程机械     6
小金属      6
服饰       6
医药商业     5
        ..
黄金       2
家用电器     2
水务       2
路桥       2
空运       2
种植业      2
石油加工     2
百货       2
电信运营     2
房产服务     2
汽车整车     2
机床制造     2
机场       2
摩托车      2
旅游服务     2
新型电力     2
矿物制品     1
轻工机械     1
啤酒       1
船舶       1
纺织机械     1
商品城      1
林业       1
染料涂料     1
特种钢      1
渔业       1
汽车服务     1
日用化工     1
水力发电     1
旅游景点     1
Length: 96, dtype: int64

In [230]:
industry_list = []
for i in df1.industry.values:
    res = df1[(df1['industry'] == i) & (df1['pp'] > 0)].rest_amount.sum() - df1[(df1['industry'] == i) & (df1['pp'] < 0)].rest_amount.sum()
    industry_list.append({'industry': i, 'amount': res})

In [235]:
df_deal = pd.DataFrame(industry_list)

In [251]:
df1[(df1['industry'] == '软件服务') & (df1['pp'] > 0)]

,ts_code,symbol,name,area,industry,list_date,last_3min_max_price,last_3min_vol,last_3min_amount,bidding_price,bidding_vol,bidding_amount,买卖类型,change,rest_amount,pp
412,000938.SZ,000938,紫光股份,北京,软件服务,19991104,26.94,2200.0,5919415.0,28.14,42027,118263978,卖盘,19.103182,112344563.0,4.454343
522,002065.SZ,002065,东华软件,北京,软件服务,20060823,9.04,10021.0,9044067.0,9.15,44657,40861137,卖盘,4.456342,31817070.0,1.216814
686,002230.SZ,002230,科大讯飞,安徽,软件服务,20080512,33.09,5131.0,16973471.0,33.22,29668,98557096,卖盘,5.782109,81583625.0,0.392868
864,002410.SZ,002410,广联达,北京,软件服务,20100525,34.38,3048.0,10458770.0,34.69,44728,155162195,卖盘,14.674541,144703425.0,0.901687
1428,300033.SZ,300033,同花顺,浙江,软件服务,20091225,90.88,1716.0,15582181.0,92.00,9386,86351200,卖盘,5.469697,70769019.0,1.232394
1644,300253.SZ,300253,卫宁健康,上海,软件服务,20110818,15.29,7854.0,11977751.0,15.38,48079,73945502,卖盘,6.121594,61967751.0,0.588620
1843,300454.SZ,300454,深信服,深圳,软件服务,20180516,118.13,744.0,8755301.0,119.54,9275,110867732,卖盘,12.466398,102112431.0,1.193600
2612,600536.SH,600536,中国软件,北京,软件服务,20020517,67.43,3065.0,20654161.0,68.30,20461,139748630,卖盘,6.675693,119094469.0,1.290227
2640,600570.SH,600570,恒生电子,浙江,软件服务,20031216,73.47,1577.0,11577837.0,74.38,11340,84346920,卖盘,7.190869,72769083.0,1.238601
2657,600588.SH,600588,用友网络,北京,软件服务,20010518,27.77,3834.0,10634031.0,28.10,28553,80233930,卖盘,7.447314,69599899.0,1.188333


In [255]:
df1[df1['industry'] == '元器件'].sort_values(by= 'pp',ascending=False)

,ts_code,symbol,name,area,industry,list_date,last_3min_max_price,last_3min_vol,last_3min_amount,bidding_price,bidding_vol,bidding_amount,买卖类型,change,rest_amount,pp
917,002463.SZ,002463,沪电股份,江苏,元器件,20100818,20.78,10192.0,21165012.0,21.16,70809,149831844,卖盘,6.947508,128666832.0,1.828681
1052,002600.SZ,002600,领益智造,广东,元器件,20110715,10.28,20026.0,20549986.0,10.46,206550,216050777,卖盘,10.314092,195500791.0,1.750973
506,002049.SZ,002049,紫光国微,河北,元器件,20050606,43.41,4270.0,18462850.0,44.10,25998,114651180,卖盘,6.088525,96188330.0,1.589496
1797,300408.SZ,300408,三环集团,广东,元器件,20141203,17.22,1069.0,1839008.0,17.46,17731,30958431,卖盘,16.586529,29119423.0,1.393728
3276,603160.SH,603160,汇顶科技,深圳,元器件,20161017,190.09,1257.0,23868624.0,192.64,9897,190655808,卖盘,7.873508,166787184.0,1.341470
697,002241.SZ,002241,歌尔股份,山东,元器件,20080522,18.66,11306.0,21059226.0,18.88,67912,128217856,卖盘,6.006722,107158630.0,1.178992
1822,300433.SZ,300433,蓝思科技,湖南,元器件,20150318,12.24,13427.0,16404380.0,12.38,62625,77529750,卖盘,4.664110,61125370.0,1.143791
2329,600183.SH,600183,生益科技,广东,元器件,19981028,21.48,8029.0,17200034.0,21.72,76146,165389112,卖盘,9.483871,148189078.0,1.117318
635,002179.SZ,002179,中航光电,河南,元器件,20071101,38.66,3053.0,11780697.0,39.00,41936,163550400,卖盘,13.735997,151769703.0,0.879462
1935,300548.SZ,300548,博创科技,浙江,元器件,20161012,61.18,262.0,1586872.0,61.70,468,2887560,买盘,1.786260,1300688.0,0.849951


In [256]:
df1.to_csv('2019-11-26尾盘异动股票.csv', encoding='utf-8-sig')